In [1]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Data preparation

In [22]:

data = pd.read_csv('induce-data-2019-08-08.csv').iloc[:, :]
vocab = ['C_E_F_T',
         'C_E_F_C',
         'C_E_F_O',
         'A_E_F_T',
         'A_E_F_O',
         'A_E_F_C',
         'G_E_F_C',
         'G_E_F_T',
         'G_E_F_O',
         'A_E_M_T',
         'A_E_M_O',
         'A_E_M_C',
         'G_E_M_O',
         'G_E_M_C',
         'G_E_M_T',
         'C_E_M_O',
         'C_E_M_C',
         'C_E_M_T',
         'C_H_F_CO',
         'C_H_F_CT',
         'C_H_F_OT',
         'G_H_F_OT',
         'G_H_F_CO',
         'G_H_F_CT',
         'A_H_F_CT',
         'A_H_F_OT',
         'A_H_F_CO',
         'C_H_M_CO',
         'C_H_M_CT',
         'C_H_M_OT',
         'A_H_M_CT',
         'A_H_M_OT',
         'A_H_M_CO',
         'G_H_M_OT',
         'G_H_M_CO',
         'G_H_M_CT', ]

labels = ['correct',
          'wrong',
          'type',
          'orientation',
          'color']

types = ['INTRO',
         'CORE',
         'FLEX',
         'TRIK',
         'DELY'
]

feat = ['type',
        'color',
        'orientation',
        'dual'
]

qts_id = ['A_E_F_C_INTRO', 'A_E_F_O_INTRO', 'A_E_F_T_INTRO', 'A_E_M_C_INTRO',
        'A_E_M_O_INTRO', 'A_E_M_T_INTRO', 'A_H_F_CO_CORE', 'A_H_F_CO_DELY',
        'A_H_F_CO_TRIK', 'A_H_F_CT_CORE', 'A_H_F_CT_DELY', 'A_H_F_CT_TRIK',
        'A_H_F_OT_CORE', 'A_H_F_OT_DELY', 'A_H_F_OT_TRIK', 'A_H_M_CO_CORE',
        'A_H_M_CO_FLEX', 'A_H_M_CT_CORE', 'A_H_M_CT_FLEX', 'A_H_M_OT_CORE',
        'A_H_M_OT_FLEX', 'C_E_F_C_INTRO', 'C_E_F_O_INTRO', 'C_E_F_T_INTRO',
        'C_E_M_C_INTRO', 'C_E_M_O_INTRO', 'C_E_M_T_INTRO', 'C_H_F_CO_CORE',
        'C_H_F_CO_DELY', 'C_H_F_CO_TRIK', 'C_H_F_CT_CORE', 'C_H_F_CT_DELY',
        'C_H_F_CT_TRIK', 'C_H_F_OT_CORE', 'C_H_F_OT_DELY', 'C_H_F_OT_TRIK',
        'C_H_M_CO_CORE', 'C_H_M_CO_FLEX', 'C_H_M_CT_CORE', 'C_H_M_CT_FLEX',
        'C_H_M_OT_CORE', 'C_H_M_OT_FLEX', 'G_E_F_C_INTRO', 'G_E_F_O_INTRO',
        'G_E_F_T_INTRO', 'G_E_M_C_INTRO', 'G_E_M_O_INTRO', 'G_E_M_T_INTRO',
        'G_H_F_CO_CORE', 'G_H_F_CO_DELY', 'G_H_F_CO_TRIK', 'G_H_F_CT_CORE',
        'G_H_F_CT_DELY', 'G_H_F_CT_TRIK', 'G_H_F_OT_CORE', 'G_H_F_OT_DELY',
        'G_H_F_OT_TRIK', 'G_H_M_CO_CORE', 'G_H_M_CO_FLEX', 'G_H_M_CT_CORE',
        'G_H_M_CT_FLEX', 'G_H_M_OT_CORE', 'G_H_M_OT_FLEX']




def seq_to_int(qts,vocab, labels, types, feat, qts_id, n_steps):
    integ = list()
    for i,x in enumerate(qts):
        if i != n_steps-1:
            features = list()
            features.append(vocab.index(qts[i, 4]))
            features.append(labels.index(qts[i, 5]))
            features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(qts_id.index(qts[i, 8]))
        else:
            features = list()
            features.append(vocab.index(qts[i, 4]))
            features.append(-1)
            features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(qts_id.index(qts[i, 8]))
        integ.append(features)
    return integ

def split_sequence(data, n_steps, vocab, labels, types, feat, qts_id):
    X, Y = list(), list()
    users = list(dict.fromkeys(data.loc[:, "user"]))
    for u in users:
        sequence = data[data.user == u]
        for i in range(len(sequence)):
            end_idx = i + n_steps
            if end_idx > len(sequence)-1:
                break
            x = seq_to_int(sequence.iloc[i:end_idx, :].values, vocab, labels, types, feat, qts_id, n_steps)
            y = labels.index(str(sequence.iloc[end_idx, 5]))
            X.append(x)
            Y.append(y)
    return np.array(X), np.array(Y)


### Test split_sequence


In [24]:
print(split_sequence(data.iloc[:20, :], 10, vocab, labels, types, feat, qts_id))


(array([[[ 0,  0,  0,  0,  0, 23],
        [ 1,  0,  1,  0,  1, 21],
        [ 2,  0,  2,  0,  2, 22],
        [ 3,  0,  3,  0,  0,  2],
        [ 4,  0,  4,  0,  2,  1],
        [ 5,  1,  5,  0,  1,  0],
        [ 6,  0,  6,  0,  1, 42],
        [ 7,  0,  7,  0,  0, 44],
        [ 8,  0,  8,  0,  2, 43],
        [ 9, -1,  9,  0,  0,  5]],

       [[ 1,  0,  1,  0,  1, 21],
        [ 2,  0,  2,  0,  2, 22],
        [ 3,  0,  3,  0,  0,  2],
        [ 4,  0,  4,  0,  2,  1],
        [ 5,  1,  5,  0,  1,  0],
        [ 6,  0,  6,  0,  1, 42],
        [ 7,  0,  7,  0,  0, 44],
        [ 8,  0,  8,  0,  2, 43],
        [ 9,  0,  9,  0,  0,  5],
        [10, -1, 10,  0,  2,  4]],

       [[ 2,  0,  2,  0,  2, 22],
        [ 3,  0,  3,  0,  0,  2],
        [ 4,  0,  4,  0,  2,  1],
        [ 5,  1,  5,  0,  1,  0],
        [ 6,  0,  6,  0,  1, 42],
        [ 7,  0,  7,  0,  0, 44],
        [ 8,  0,  8,  0,  2, 43],
        [ 9,  0,  9,  0,  0,  5],
        [10,  0, 10,  0,  2,  4],
        [

## The model


In [25]:
n_features = 6
n_steps = 10


X, y = split_sequence(data, n_steps, vocab, labels, types, feat, qts_id)
y = np_utils.to_categorical(y)
X = X.reshape((X.shape[0], X.shape[1], n_features))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
print('train set size: '+str(len(y_train)))
print('test set size: '+str(len(y_test)))

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse', metrics=['categorical_accuracy'])

train set size: 10572
test set size: 1175


### Training

In [27]:
# fit model
model.fit(X_train, y_train, epochs=2000, verbose=2)

Epoch 1/2000
 - 14s - loss: 0.0955 - categorical_accuracy: 0.6153
Epoch 2/2000
 - 12s - loss: 0.0919 - categorical_accuracy: 0.6274
Epoch 3/2000
 - 11s - loss: 0.0910 - categorical_accuracy: 0.6368
Epoch 4/2000
 - 11s - loss: 0.0907 - categorical_accuracy: 0.6318
Epoch 5/2000
 - 11s - loss: 0.0898 - categorical_accuracy: 0.6344
Epoch 6/2000
 - 13s - loss: 0.0893 - categorical_accuracy: 0.6385
Epoch 7/2000
 - 17s - loss: 0.0891 - categorical_accuracy: 0.6343
Epoch 8/2000
 - 13s - loss: 0.0890 - categorical_accuracy: 0.6366
Epoch 9/2000
 - 11s - loss: 0.0887 - categorical_accuracy: 0.6403
Epoch 10/2000
 - 11s - loss: 0.0882 - categorical_accuracy: 0.6393
Epoch 11/2000
 - 11s - loss: 0.0879 - categorical_accuracy: 0.6387
Epoch 12/2000
 - 11s - loss: 0.0879 - categorical_accuracy: 0.6371
Epoch 13/2000
 - 11s - loss: 0.0876 - categorical_accuracy: 0.6415
Epoch 14/2000
 - 11s - loss: 0.0876 - categorical_accuracy: 0.6442
Epoch 15/2000
 - 11s - loss: 0.0878 - categorical_accuracy: 0.6433
Epoc

### Validation

In [28]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


print(X_test[0])
print(model.predict(np.array([X_test[0]])))
print(np.array([X_test[1]]))
print(y_test[0])



Accuracy: 64.26%
[[ 9  0  9  0  0  5]
 [10  0 10  0  2  4]
 [11  0 11  0  1  3]
 [12  0 12  0  2 46]
 [13  0 13  0  1 45]
 [14  0 14  0  0 47]
 [15  0 15  0  2 25]
 [16  0 16  0  1 24]
 [17  0 17  0  0 26]
 [18 -1 18  1  3 27]]
[[ 1.9427319e-04  6.9832280e-03  1.0266405e+00 -2.8348029e-02
  -8.5514411e-03]]
[[[27  4 39  2  3 37]
  [24  2 40  3  3 11]
  [30  2 41  2  3 18]
  [21  2 42  4  3 55]
  [18  4 43  4  3 28]
  [24  2 44  4  3 10]
  [25  2 45  3  3 14]
  [31  3 46  2  3 20]
  [19  4 47  3  3 32]
  [28 -1 48  2  3 39]]]
[0. 0. 1. 0. 0.]
